In [33]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [31]:
train = pd.read_csv('datasets/hwu/train.csv')
val = pd.read_csv('datasets/hwu/val.csv')
test = pd.read_csv('datasets/hwu/test.csv')

In [ ]:
train

,text,category
0,what alarms do i have set right now,alarm_query
1,checkout today alarm of meeting,alarm_query
2,report alarm settings,alarm_query
3,see see for me the alarms that you have set to...,alarm_query
4,is there an alarm for ten am,alarm_query
...,...,...
8949,how hot is it in miami,weather_query
8950,will it snow next week,weather_query
8951,am i gonna need rain boots,weather_query
8952,should i bring warm clothes,weather_query


In [16]:
def tokenizer(text):
    tokens = text.split(' ')
    return tokens

In [48]:
train_tokens = train['text'].apply(tokenizer).to_list()
test_tokens = test['text'].apply(tokenizer).tolist()
print(len(train_tokens))
print(max(len(train_tokens[i]) for i in range(8954)))

8954
25


In [68]:
le = LabelEncoder()
train_labels = le.fit_transform(train['category'])
test_labels = le.transform(test['category'])
len(set(train_labels))

64

In [61]:
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

In [49]:
indexer = Tokenizer(oov_token="<OOV>")
indexer.fit_on_texts(train_tokens)
train_sequences = indexer.texts_to_sequences(train_tokens)
test_sequences = indexer.texts_to_sequences(test_tokens)
train_padded_seq = pad_sequences(train_sequences, maxlen=25, padding='post', truncating='post')
test_padded_seq = pad_sequences(test_sequences, maxlen=25, padding='post', truncating='post')

In [74]:
vocab_size = len(indexer.word_index) + 1
embedding_dim = 64

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim),
    LSTM(units=64, return_sequences=False),
    Dense(64, activation='softmax'),
])

In [72]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(train_padded_seq, train_labels, epochs=10, validation_data=(test_padded_seq, test_labels), batch_size=32)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node sequential_1_1/lstm_1_1/while/TensorListPushBack_13 defined at (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main

  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code

  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>

  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "C:\Users\LENOVO\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 600, in run_forever

  File "c:\Use

In [70]:
print(type(train_padded_seq), train_padded_seq.shape)
print(type(train_labels), train_labels.shape)

<class 'numpy.ndarray'> (8954, 25)
<class 'numpy.ndarray'> (8954,)


In [77]:
print(train_padded_seq.shape, test_padded_seq.shape)
print(train_labels.shape, test_labels.shape)
print(train_padded_seq[0])

(8954, 25) (1076, 25)
(8954,) (1076,)
[  8  96  24   5  25  35 119  65   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0]
